In [ ]:
!pip install kaggle

Importing the dependencies

In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.2 MB/s eta 0:00:00


In [1]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Bidirectional, BatchNormalization, Dropout

Data Collection

In [2]:
kaggle_dictionary = json.load(open('kaggle.json'))
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [3]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other


In [4]:
from zipfile import ZipFile
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
    zip_ref.extractall()


In [5]:
df = pd.read_csv("/content/IMDB Dataset.csv") # path to your data

In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [8]:
df.replace({"sentiment":{"positive":1, "negative":0}}, inplace = True)

<ipython-input-8-e747cd52c10c>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment":{"positive":1, "negative":0}}, inplace = True)


In [9]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [10]:
train_data , test_data = train_test_split(df, test_size=0.25, random_state=42, shuffle= True)

In [11]:
print(train_data.shape)
print(test_data.shape)

(37500, 2)
(12500, 2)


In [12]:
tokenizer = Tokenizer(num_words=5000) # converts word to numbers
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen= 200)


In [13]:
print(X_train)

[[   0    0    0 ...    1   13  190]
 [ 148   48    1 ...  103    3  577]
 [   0    0    0 ...  292   29 2104]
 ...
 [   0    0    0 ... 1614    2  593]
 [   0    0    0 ...  246  103  125]
 [   0    0    0 ...   70   72 2069]]


In [14]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]


In [15]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)


Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 324s 664ms/step - accuracy: 0.7219 - loss: 0.5303 - val_accuracy: 0.8499 - val_loss: 0.3543
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 319s 681ms/step - accuracy: 0.8550 - loss: 0.3507 - val_accuracy: 0.8664 - val_loss: 0.3242
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 325s 688ms/step - accuracy: 0.8668 - loss: 0.3221 - val_accuracy: 0.8667 - val_loss: 0.3258
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 379s 682ms/step - accuracy: 0.8882 - loss: 0.2772 - val_accuracy: 0.8520 - val_loss: 0.3482
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 322s 683ms/step - accuracy: 0.8953 - loss: 0.2640 - val_accuracy: 0.8707 - val_loss: 0.3353


In [18]:
loss, accuracy = model.evaluate(X_test, Y_test)


391/391 ━━━━━━━━━━━━━━━━━━━━ 45s 114ms/step - accuracy: 0.8732 - loss: 0.3293


In [19]:
def predictive_sentiment_alanylis(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)

  if prediction[0][0] > 0.5:
    print("Positive")
  else:
    print("Negative")


In [20]:
# prompt: give me code to save the model and tokenizer using pickle

import pickle

# Save the model
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Save the tokenizer
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


In [22]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.8 MB/s eta 0:00:00


In [23]:
# prompt: give me streamlit code to deploy the model on web such that it takes user input as review and gives out result as positive or negative

import streamlit as st
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the model and tokenizer
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Define the Streamlit app
st.title("Sentiment Analysis")

# Get user input
user_review = st.text_area("Enter your movie review:")

if st.button("Analyze"):
    if user_review:
        # Preprocess the review
        sequence = tokenizer.texts_to_sequences([user_review])
        padded_sequence = pad_sequences(sequence, maxlen=200)

        # Make prediction
        prediction = model.predict(padded_sequence)[0][0]

        # Display the result
        if prediction > 0.5:
            st.success("Positive Sentiment")
        else:
            st.error("Negative Sentiment")
    else:
        st.warning("Please enter a review.")


2025-03-21 21:52:54.906 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-21 21:52:55.008 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-03-21 21:52:55.010 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-21 21:52:55.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-21 21:52:55.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-21 21:52:55.014 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-21 21:52:55.015 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-21 21:52:55.017 Session state does not 